In [2]:

# Let's first install Cohere's python SDK
# %pip install cohere
# %pip install python-dotenv

In [3]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

import os
from dotenv import load_dotenv

load_dotenv()
# Paste your API key here. Remember to not share publicly
api_key = os.getenv('COHERE_API_KEY')

In [5]:
# load the dataset
df = pd.read_csv('../data/Example_data.xlsx')

In [6]:
# Let's glance at the dataset
df.head()

Domain  \
0         rassegnastampa.news   
1                 twitter.com   
2             atpe-tchad.info   
3  badbluetech.bitnamiapp.com   
4           kryptogazette.com   

                                                                                                                                                                                                                                                                         Title  \
0                                                                                                                                                    Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for   
1  Stumbled across an interesting case, a woman facing eviction from the home that she inherited from her brother in 2007. Turns out her brother was in business with a notorious international criminal Paul Le Roux, who is currently serving 25 years in a New York prison.   
2                                               Marché Résines dans les peintures et revêtements 2021 avec les données des meilleurs pays et l’analyse Covid-19, la portée future, l’estimation de la taille, les revenus, les tendances des prix et les prévisions d’ici 2026   
3                                                                                                                                                                                                                                  AI drives data analytics surge, study finds   
4                                                                                                                 Triacetin Vertrieb Markt 2021: Globale Unternehmensanalyse, Merkmale, Marktplatzlänge und -prognosen bis zu 2027 mit Hauptregionen und Nationeninformationen   

                                                                                                                                                                                                                                                                   Description  \
0                  …often trigger a protest vote that can upset…that ministers who breach the Ministerial Code…plane up the M1 in order to…serious questions about abuse of taxpayers' money which is a clear breach of the code.…should not permit abuse of taxpayers' money…   
1  Stumbled across an interesting case, a woman facing eviction from the home that she inherited from her brother in 2007. Turns out her brother was in business with a notorious international criminal Paul Le Roux, who is currently serving 25 years in a New York prison.   
2                        …COVID-19…COVID…COVID…COVID-19 et Post COVID…COVID…Covid…BASF…COVID…COVID-19 3.4.1 Évaluation des risques liés au COVID-19 3.4.2 Évaluation de l’impact global du COVID-19 sur l’industrie 3.4.3 Scénario de marché avant et après COVID…covid…covid…   
3                     …hate raiders' linked to automated harassment campaigns [engadget](d.php?did=engadget.com) Texas law could force social media to host misinformation and hate speech [ars](d.php?did=arstechnica.com)…breach…shot on…stolen data…Killed…M1…Toxic…Attack…   
4                  …Abschnitten und Endanwendungen / Organisationen. Covid-19 kann die Weltwirtschaft auf…/ Key-Player profiliert: Eastman Polynt Lanxess BASF Daicel Jiangsu Ruijia Jiangsu Lemon…Japan, wie Eastman, Lanxess, BASF, Daicel und Jiangsu Ruijia konzentriert.…   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
num_examples = 500
df_sample = df.sample(num_examples)

# Split into training and testing sets
sentences_train, sentences_test, labels_train, labels_test = train_test_split(
            list(df_sample[0]), list(df_sample[1]), test_size=0.25, random_state=0)